In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import spacy
import nltk
import gensim
from collections import Counter
from string import punctuation

In [5]:
import nltk
from nltk.corpus import stopwords

# EDA #

In [6]:
# Import Dataframes
s_script = pd.read_csv('simpsons_data/simpsons_script_lines.csv')
s_char = pd.read_csv('simpsons_data/simpsons_characters.csv')
s_ep = pd.read_csv('simpsons_data/simpsons_episodes.csv') 
s_loc = pd.read_csv('simpsons_data/simpsons_locations.csv')

/Users/justincarloulim/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
# Create a dataframe that merges both script and the character information.
# Now, we have the gender of each character, as well as their names, the timestamp
# of their speaking lines, and their character id.
script = s_script[['episode_id', 
                   'timestamp_in_ms', 
                   'raw_character_text', 
                   'spoken_words', 
                   'word_count']].rename(columns = {'raw_character_text': 'name', 
                                                    'spoken_words': 'line'})
chars = s_char.dropna().rename(columns = {'id': 'char_id'})
script_table = script.merge(chars)
script_table = script_table[['episode_id', 'normalized_name', 'char_id', 'line', 'timestamp_in_ms', 'gender']]
script_table.head()

,episode_id,normalized_name,char_id,line,timestamp_in_ms,gender
0,32,miss hoover,464,"No, actually, it was a little of both. Sometim...",848000,f
1,32,miss hoover,464,I don't know. Although I'd sure like to talk t...,856000,f
2,49,miss hoover,464,"Thank you, Ralph. Very graphic. Lisa Simpson? ...",784000,f
3,49,miss hoover,464,Dear God!,803000,f
4,51,miss hoover,464,I question the educational value of this assem...,226000,f


# TF-IDF Model #

# Network Analysis Model #

# Topic Model #

In [44]:
spacy.load('en_core_web_sm')
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

parser = English()
en_stop = set(nltk.corpus.stopwords.words('english'))

In [85]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        elif type(token) != str:
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = text.lower()
    for char in punctuation:
        tokens = tokens.replace(char, '')
    tokens = tokens.split()
    return tokens

In [114]:
#script_table['line']

In [103]:
# Lower and remove punctuation from each line, and put
# Into a 2d list
text_data = []
for line in script_table['line']:
    if type(line) != str:
        continue
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)

111314

In [115]:
quarter_index = int(len(text_data)/4)
text_data_quarter = text_data[: quarter_index]

In [112]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data_quarter]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [113]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.045*"i" + 0.043*"the" + 0.032*"a" + 0.027*"to"')
(1, '0.026*"okay" + 0.014*"gotta" + 0.009*"kid" + 0.007*"thank"')
(2, '0.036*"oh" + 0.027*"no" + 0.020*"my" + 0.020*"im"')
(3, '0.061*"you" + 0.031*"what" + 0.028*"do" + 0.027*"hey"')
(4, '0.022*"uh" + 0.008*"a" + 0.008*"cool" + 0.007*"um"')
